In [43]:
import numpy as np
import pandas as pd
from pulp import *
from geopy.distance import distance

In [72]:
wells = pd.read_csv("data/wells/AllWells_20210915.csv")
wells_oilgas = wells[wells['WellTypeLa']== 'Oil & Gas']
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_ids = wells_oilgas['API'].tolist()

In [83]:
wells_oilgas

,OID,API,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,FieldName,...,Township,Range,BaseMeridi,Latitude,Longitude,GISSource,isConfiden,isDirectio,SpudDate,WellSymbol
53,-1,408300962,Alegria,1,Plugged,OG,Oil & Gas,01279,Cal-L Expl. Corp.,Alegria (ABD),...,05N,32W,SB,34.471405,-120.271706,hud,N,N,,PluggedOG
54,-1,408300972,Alegria,10,Plugged,OG,Oil & Gas,01279,Cal-L Expl. Corp.,Alegria (ABD),...,05N,32W,SB,34.469040,-120.257660,hud,N,N,,PluggedOG
55,-1,408300974,Alegria,14,Plugged,OG,Oil & Gas,01279,Cal-L Expl. Corp.,Alegria (ABD),...,05N,32W,SB,34.474018,-120.254334,hud,N,N,,PluggedOG
56,-1,408300976,Alegria,2,Plugged,OG,Oil & Gas,01279,Cal-L Expl. Corp.,Alegria (ABD),...,05N,32W,SB,34.472298,-120.271194,hud,N,N,,PluggedOG
57,-1,408300968,Alegria,3,Plugged,OG,Oil & Gas,01279,Cal-L Expl. Corp.,Alegria (ABD),...,05N,32W,SB,34.469498,-120.264503,hud,N,N,,PluggedOG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240863,-1,408301011,Quati,98,Active,OG,Oil & Gas,G3515,"HVI Cat Canyon, Inc.",Zaca,...,07N,31W,SB,34.720535,-120.127739,GPS,N,N,,ActiveOG
240864,-1,408304478,Quati,1,Plugged,OG,Oil & Gas,G3515,"HVI Cat Canyon, Inc.",Zaca,...,08N,31W,SB,34.725594,-120.154022,hud,N,N,,PluggedOG
240865,-1,408301012,Quati,75,Plugged,OG,Oil & Gas,G3515,"HVI Cat Canyon, Inc.",Zaca,...,08N,31W,SB,34.725861,-120.131943,hud,N,N,,PluggedOG
240866,-1,408301013,Quati,77,Plugged,OG,Oil & Gas,G3515,"HVI Cat Canyon, Inc.",Zaca,...,07N,31W,SB,34.722115,-120.131737,hud,N,N,,PluggedOG


In [23]:
emissions = pd.read_csv(r'./data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'./data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
facilities = facilities[facilities.State == 'CA']

In [36]:
facilitiesB = facilities.rename(inplace=False,
                                columns={'Facility Name':'facility_name',
                                         'City':'city',
                                         'Primary NAICS Code':'naics_code',
                                         'Industry Type (subparts)':'industry_subparts',
                                         'Industry Type (sectors)':'industry_sectors'})

emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
#                                               'Facility Name2',
                                               'city',
                                               'Zip Code',
                                               'Address',
                                               'County',
                                               'Latitude',
                                               'Longitude',
#                                               'Primary NAICS Code2',
                                               'industry_subparts',
                                               'industry_sectors']],
                        how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','Latitude','Longitude','Unit CO2 emissions (non-biogenic)']].dropna().groupby('Facility Id').agg({'Latitude':'min',
                                                           'Longitude':'min',
                                                           'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                  'Latitude':'lat',
                                                  'Longitude':'lon',
                                                  'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg.head()


In [84]:
facility_latlon = emsCA2019locagg[['lat', 'lon']].to_records(index=False)
facility_ids = emsCA2019locagg['facility_id'].tolist()

In [ ]:
# This cell takes ages to run - it's creating a full cartesian matrix between facilities and wells

distance_matrix = []
for i in wells_latlon:
    distances = []
    for j in facility_latlon:
        dist = distance(i,j).miles
        distances.append(dist)
    distance_matrix.append(distances)
    
print(distance_matrix)

In [81]:
prob = LpProblem('Unbalaced Transportation Problem', LpMinimize)
n_receivers = len(wells_ids)
n_suppliers = len(facility_ids)
supply = emsCA2019locagg[['annual_emmissions']].tolist()
demand = None #NEED WELL VOLUMES HERE

routes = [(i, j) for i in range(n_suppliers) for j in range(n_receivers)]

x = LpVariable.dicts('X', routes, lowBound=0)
prob += lpSum([x[i, j] * costs[i][j] for i in range(n_suppliers) for j in range(n_receivers)])

for i in range(n_suppliers):
    prob += lpSum([x[i, j] for j in range(n_receivers)]) == supply[i]
    
for j in range(n_receivers):
    prob += lpSum([x[i, j] for i in range(n_suppliers)]) <= demand[j]
    
# Solving problem
prob.solve()
print('Status', LpStatus[prob.status])

print('Z = {}'.format(value(prob.objective)))
for v in prob.variables():
    print('{} = {}'.format(v.name, v.varValue))

SyntaxError: invalid syntax (<ipython-input-81-e9ef31dfe42c>, line 5)